In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import random
import glob
import os
from pandas import json_normalize

In [8]:
# Path to the folder containing JSON files
path_to_json = 'jsons/'

# List all JSON files in the folder
json_files = glob.glob(os.path.join(path_to_json, '*.json'))

# List to store DataFrames from each JSON file
data_frames = []

In [12]:
# Process each JSON file
for file in json_files:
    # Read the JSON file
    df = pd.read_json(file)
    
    # Normalize the 'messages' section
    df = json_normalize(df['messages'])
    
    # Extract the channel name
    channel_name = df['from'].dropna().iloc[0] if 'from' in df.columns else 'Unknown Channel'
    
    # Select and rename necessary columns
    df = df[['date', 'text_entities']]
    df['channel_name'] = channel_name  # Add the channel name as a new column
    
    # Filter messages with non-empty 'text_entities'
    df = df[df['text_entities'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
    
    # Extract message text from 'text_entities'
    def extract_message(text_entities):
        try:
            return ''.join([entity['text'] for entity in text_entities if 'text' in entity])
        except:
            return None
    
    # Apply the message extraction function
    df['message'] = df['text_entities'].apply(extract_message)
    
    # Drop 'text_entities' as it's no longer needed
    df = df.drop(columns=['text_entities'])
    
    # Append the processed DataFrame to the list
    data_frames.append(df)

# Concatenate all DataFrames into one
df = pd.concat(data_frames, ignore_index=True)

# Display the final combined DataFrame
df.head()

,date,channel_name,message
0,2020-04-26T23:10:32,podselenie_v_astane,Здравствуйте 👋
1,2020-04-26T23:10:43,podselenie_v_astane,Подписывайтесь ✔️
2,2020-04-26T23:11:29,podselenie_v_astane,В будущем в первую очередь здесь будут публико...
3,2020-04-29T19:44:41,podselenie_v_astane,Сдаю комнату в 2-х комнатной квартире по адрес...
4,2020-04-29T20:22:29,podselenie_v_astane,Здравствуйте ищем на подселение 2 девушек. Пор...


In [13]:
df =df.sort_values(by='date', ascending=False)

In [14]:
df.to_excel('datas.xlsx')

In [5]:
df['channel_name'].unique()

array(['podselenie_v_astane', 'DAILY ХАТА',
       'Квартиры Алматы | Аренда квартир Алматы | Поиск соседей Алматы',
       'КВАРТИРА АЛМАТЫ | ПӘТЕР АЛМАТЫ | НЕДВИЖИМОСТЬ АЛМАТЫ',
       'ПОДСЕЛЕНИЕ АСТАНА КВАРТИРА', 'sdam_snimu_astana',
       'Студенты Астана (аренда, подселение)'], dtype=object)

In [10]:
df.isnull().sum()

date            0
channel_name    0
message         0
dtype: int64

In [11]:
df.count()

date            51539
channel_name    51539
message         51539
dtype: int64

In [14]:
df[df['channel_name'] == 'КВАРТИРА АЛМАТЫ | ПӘТЕР АЛМАТЫ | НЕДВИЖИМОСТЬ АЛМАТЫ'].tail()

,date,channel_name,message
27732,2024-10-22T12:53:36,КВАРТИРА АЛМАТЫ | ПӘТЕР АЛМАТЫ | НЕДВИЖИМОСТЬ ...,На подселение нужна девушка 30000тг. Квартира ...
27733,2024-10-22T16:54:55,КВАРТИРА АЛМАТЫ | ПӘТЕР АЛМАТЫ | НЕДВИЖИМОСТЬ ...,Срочно!!! \nПодселениеге 1 қыз керек 1 ком.\n\...
27734,2024-10-23T00:30:04,КВАРТИРА АЛМАТЫ | ПӘТЕР АЛМАТЫ | НЕДВИЖИМОСТЬ ...,‼️СРОЧНО‼️\n2 Бөлмелі квартираға 2 адам іздейм...
27735,2024-10-25T23:54:24,КВАРТИРА АЛМАТЫ | ПӘТЕР АЛМАТЫ | НЕДВИЖИМОСТЬ ...,Ищу квартиру с хорошим ремонтом и с мебелью. \...
27736,2024-10-28T14:53:24,КВАРТИРА АЛМАТЫ | ПӘТЕР АЛМАТЫ | НЕДВИЖИМОСТЬ ...,Срочно подселениеге 2-3 кыз керек (Однокомна...


In [20]:
# df.to_csv('df.csv')
# Load the CSV file
df = pd.read_csv('df.csv')

# Drop any column named 'Unnamed' or similar that should represent 'id'
df = df.rename(columns=lambda x: 'id' if 'Unnamed' in x else x)
df

,id,date,channel_name,message
0,0,2020-04-26T23:10:32,podselenie_v_astane,Здравствуйте 👋
1,1,2020-04-26T23:10:43,podselenie_v_astane,Подписывайтесь ✔️
2,2,2020-04-26T23:11:29,podselenie_v_astane,В будущем в первую очередь здесь будут публико...
3,3,2020-04-29T19:44:41,podselenie_v_astane,Сдаю комнату в 2-х комнатной квартире по адрес...
4,4,2020-04-29T20:22:29,podselenie_v_astane,Здравствуйте ищем на подселение 2 девушек. Пор...
...,...,...,...,...
51534,51534,2024-10-22T13:48:49,"Студенты Астана (аренда, подселение)",Здравствуйте! \n\nДевушка!\nСРОЧНО!\nИЩУ отдел...
51535,51535,2024-10-24T19:29:16,"Студенты Астана (аренда, подселение)",Срочноооо‼️‼️‼️‼️\nВсем привет ! Срочно ищу 4 ...
51536,51536,2024-10-26T15:18:51,"Студенты Астана (аренда, подселение)",Возьму на подселение трех девушек до 25 лет. п...
51537,51537,2024-10-27T16:42:59,"Студенты Астана (аренда, подселение)","🏠Ищем третью девушку на подселение, 1 комнатна..."


In [21]:
df = df.tail(100)
df.head()

,id,date,channel_name,message
51439,51439,2024-09-10T16:18:15,"Студенты Астана (аренда, подселение)",Чтоб добавить своё объявление напишите:\n@Stud...
51440,51440,2024-09-10T17:42:37,"Студенты Астана (аренда, подселение)","Заселюсь в квартиру без толпы.\n21 год, мужчин..."
51441,51441,2024-09-10T17:59:51,"Студенты Астана (аренда, подселение)",1 комнаталы квартираға подселениеге 1 студент ...
51442,51442,2024-09-10T21:12:31,"Студенты Астана (аренда, подселение)","Подселениеге 2-қыз іздейміз,тазалыққа жақын,жа..."
51443,51443,2024-09-11T00:59:01,"Студенты Астана (аренда, подселение)","ищу к кому подселиться! студентка ЕНУ, желател..."


In [22]:
# Add new columns for city, area, and district, initializing with None using .loc
df.loc[:, 'city'] = None
df.loc[:, 'area'] = None
df.loc[:, 'district'] = None
df

C:\Users\Admin\AppData\Local\Temp\ipykernel_4420\2976968638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'city'] = None
C:\Users\Admin\AppData\Local\Temp\ipykernel_4420\2976968638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'area'] = None
C:\Users\Admin\AppData\Local\Temp\ipykernel_4420\2976968638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

,id,date,channel_name,message,city,area,district
51439,51439,2024-09-10T16:18:15,"Студенты Астана (аренда, подселение)",Чтоб добавить своё объявление напишите:\n@Stud...,None,None,None
51440,51440,2024-09-10T17:42:37,"Студенты Астана (аренда, подселение)","Заселюсь в квартиру без толпы.\n21 год, мужчин...",None,None,None
51441,51441,2024-09-10T17:59:51,"Студенты Астана (аренда, подселение)",1 комнаталы квартираға подселениеге 1 студент ...,None,None,None
51442,51442,2024-09-10T21:12:31,"Студенты Астана (аренда, подселение)","Подселениеге 2-қыз іздейміз,тазалыққа жақын,жа...",None,None,None
51443,51443,2024-09-11T00:59:01,"Студенты Астана (аренда, подселение)","ищу к кому подселиться! студентка ЕНУ, желател...",None,None,None
...,...,...,...,...,...,...,...
51534,51534,2024-10-22T13:48:49,"Студенты Астана (аренда, подселение)",Здравствуйте! \n\nДевушка!\nСРОЧНО!\nИЩУ отдел...,None,None,None
51535,51535,2024-10-24T19:29:16,"Студенты Астана (аренда, подселение)",Срочноооо‼️‼️‼️‼️\nВсем привет ! Срочно ищу 4 ...,None,None,None
51536,51536,2024-10-26T15:18:51,"Студенты Астана (аренда, подселение)",Возьму на подселение трех девушек до 25 лет. п...,None,None,None
51537,51537,2024-10-27T16:42:59,"Студенты Астана (аренда, подселение)","🏠Ищем третью девушку на подселение, 1 комнатна...",None,None,None


In [31]:
# Define an exponential backoff function
def exponential_backoff(retries):
    wait_time = min(2 ** retries + random.uniform(0, 1), 60)  # Cap the wait time at 60 seconds
    print(f"Waiting for {wait_time:.2f} seconds due to rate limit...")
    time.sleep(wait_time)

In [32]:
from google.api_core.exceptions import ResourceExhausted
from scripts.extract_address import gemini_address

# Set a maximum number of retries for each request
max_retries = 5

# Assuming df is already loaded and contains 'message' and 'channel_name' columns
for index, row in df.iterrows():
    message = row['message']  # Get the message content
    channel_name = row['channel_name']  # Get the channel name
    
    retries = 0
    success = False
    
    # Attempt to call gemini_address with retry on rate limit errors
    while retries < max_retries and not success:
        try:
            # Call gemini_address to extract city, area, and district
            result = gemini_address(message, channel_name)
            
            # Update the DataFrame with extracted values
            df.at[index, 'city'] = result.get("city")
            df.at[index, 'area'] = result.get("area")
            df.at[index, 'district'] = result.get("district")
            
            success = True  # Mark success to exit retry loop
            
        except ResourceExhausted:
            print(f"Rate limit exceeded. Attempt {retries + 1} of {max_retries}. Retrying...")
            exponential_backoff(retries)
            retries += 1
            
    if not success:
        print(f"Failed to process row {index} after {max_retries} attempts due to rate limits.")

Rate limit exceeded. Attempt 1 of 5. Retrying...
Waiting for 1.99 seconds due to rate limit...
Rate limit exceeded. Attempt 1 of 5. Retrying...
Waiting for 1.60 seconds due to rate limit...
Rate limit exceeded. Attempt 1 of 5. Retrying...
Waiting for 1.95 seconds due to rate limit...
Rate limit exceeded. Attempt 1 of 5. Retrying...
Waiting for 1.39 seconds due to rate limit...
Rate limit exceeded. Attempt 2 of 5. Retrying...
Waiting for 2.63 seconds due to rate limit...
Rate limit exceeded. Attempt 3 of 5. Retrying...
Waiting for 4.40 seconds due to rate limit...
Rate limit exceeded. Attempt 1 of 5. Retrying...
Waiting for 1.99 seconds due to rate limit...
Rate limit exceeded. Attempt 2 of 5. Retrying...
Waiting for 2.25 seconds due to rate limit...
Rate limit exceeded. Attempt 3 of 5. Retrying...
Waiting for 4.51 seconds due to rate limit...
Rate limit exceeded. Attempt 4 of 5. Retrying...
Waiting for 8.07 seconds due to rate limit...
Rate limit exceeded. Attempt 5 of 5. Retrying...
W

In [33]:
df.head()

,id,date,channel_name,message,city,area,district
51439,51439,2024-09-10T16:18:15,"Студенты Астана (аренда, подселение)",Чтоб добавить своё объявление напишите:\n@Stud...,Астана,None,None
51440,51440,2024-09-10T17:42:37,"Студенты Астана (аренда, подселение)","Заселюсь в квартиру без толпы.\n21 год, мужчин...",Астана,ЕНУ,None
51441,51441,2024-09-10T17:59:51,"Студенты Астана (аренда, подселение)",1 комнаталы квартираға подселениеге 1 студент ...,Астана,ЕНУ,None
51442,51442,2024-09-10T21:12:31,"Студенты Астана (аренда, подселение)","Подселениеге 2-қыз іздейміз,тазалыққа жақын,жа...",Астана,"Ену, Жирентаева 15/1",None
51443,51443,2024-09-11T00:59:01,"Студенты Астана (аренда, подселение)","ищу к кому подселиться! студентка ЕНУ, желател...",Астана,Алматинском/Байконурском,None


In [34]:
df.to_excel('df.xlsx')